# Introduction to Data Science in Python - Hypothesis Testing


Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as <span style = 'color:red'>starting with two consecutive quarters of GDP decline</span>, and <span style = 'color:red'>ending with two consecutive quarters of GDP growth</span>.
* A _recession bottom_ is the <span style = 'color:red'>quarter</span> within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less affected by recessions. Run a t-test to compare the ratio of the <span style = 'color:red'>mean price</span> of houses in university towns <span style = 'color:red'>the quarter before the recession starts</span> compared to the <span style = 'color:red'>recession bottom</span>. (`price_ratio = quarter_before_recession / recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has <span style = 'color:red'>median home sale prices</span> at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (<span style = 'color:red'>use the chained value in 2009 dollars</span>), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data <span style = 'color:red'>from the first quarter of 2000 onward</span>.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

<br>

In [1]:
import pandas as pd
import numpy as np
import re
from scipy.stats import ttest_ind

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 
          'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 
          'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 
          'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 
          'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 
          'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 
          'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 
          'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 
          'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 
          'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 
          'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 
          'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 
          'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 
          'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 
          'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 
          'ND': 'North Dakota', 'VA': 'Virginia'}

<br>
### Auxiliary functions:

<span style="color:blue">Very ugly way (seems faster though) to get the states and towns:</span>

In [3]:
########### Extract state names. ###########
fh = open("university_towns.txt")

lst = list()
for line in fh:
    line = line.strip()
    x = re.findall('.+(?=\[edit\])', line) 
    if len(x) > 0 : 
        # Do NOT use append(), otherwise the state names added into the list would be 
        # sublists rather than strings.
        lst.extend(x) 



########### Extract college town names. ###########
# Must reopen the text file.
fh = open("university_towns.txt")
text = fh.read()

# Split the text by state and store the content in a list. 
# Note that the first state name is kept in the data since it has no preceding "\n".
split_by_state = re.compile('\n.+\[edit\]').split(text)

# Within each state, separate each college towns. 
# Note that because we have a redundant blank line at the end of the text file, the last 
# state Wyoming would need extra processing in the end.
b = pd.Series(split_by_state).apply(lambda x: x.split("\n"))
# While b is a series, the elements of b are all lists.

# Create a wide dataframe with the first column storing the state names and subsequent 
# columns college towns.
state_region = b.apply(lambda x: pd.Series(x))


########### Match towns with states and reshape the data from wide to long ###########
state_region.iloc[:,0] = lst
state_region = (pd.melt(state_region, id_vars = [0])
                .dropna()
                .drop('variable', axis = 1)
                .rename(columns = {0:'State', 'value': 'RegionName'})
                .sort_values(['State', 'RegionName']))

# Remove the university names.
state_region.RegionName = state_region.RegionName.apply(lambda x: re.sub(r' \(.*', '', x))

# Extra processing for Wyoming.
state_region = state_region.where(state_region.RegionName != "").dropna()

state_region

,State,RegionName
0,Alabama,Auburn
50,Alabama,Florence
100,Alabama,Jacksonville
150,Alabama,Livingston
200,Alabama,Montevallo
250,Alabama,Troy
300,Alabama,Tuscaloosa
350,Alabama,Tuskegee
1,Alaska,Fairbanks
2,Arizona,Flagstaff


<br>

<span style="color:blue">Read GDP data:</span>    
In this function, read_excel() argument "skiprows" can be altered for testing the code.

In [4]:
def read_GDP():
    
    GDP = pd.read_excel('gdplev.xls', skiprows = 219, parse_cols = 'E,G',
                        names= ['quarter', 'billions_2009dollars'])
    GDP['quarterly_diff'] = GDP['billions_2009dollars'].diff()
    GDP = GDP.assign(indicator = np.where(GDP['quarterly_diff'] >= 0, 1, -1))
    
    return GDP


In [5]:
read_GDP()

,quarter,billions_2009dollars,quarterly_diff,indicator
0,2000q1,12359.1,NaN,-1
1,2000q2,12592.5,233.4,1
2,2000q3,12607.7,15.2,1
3,2000q4,12679.3,71.6,1
4,2001q1,12643.3,-36.0,-1
5,2001q2,12710.3,67.0,1
6,2001q3,12670.1,-40.2,-1
7,2001q4,12705.3,35.2,1
8,2002q1,12822.3,117.0,1
9,2002q2,12893.0,70.7,1


<br>

<span style="color:blue">Find all the quarters in which a recession starts or ends:</span>

In [6]:
def find_start_end():
    
    GDP = read_GDP()
    
    dt = pd.DataFrame()
    i = 1
    while i <= GDP.shape[0] - 2:
        if GDP.indicator[i] == -1 and GDP.indicator[i + 1] == -1:
            start = GDP.quarter[i]
            j = i + 2
            while j <= GDP.shape[0] - 2:
                if GDP.indicator[j] == 1 and GDP.indicator[j + 1] == 1:
                    end = GDP.quarter[j + 1]
                    i = j + 2
                    break
                else:
                    j = j + 1
            h = pd.Series([start, end])
            dt = dt.append(h, ignore_index = True)
        else:
            i = i + 1
    
    dt.rename(columns = {0:'start_quarter', 1:'end_quarter'}, inplace = True)
    
    return dt              

In [7]:
find_start_end()

,start_quarter,end_quarter
0,2008q3,2009q4


<br>

<span style="color:blue">Return all the recession bottoms:</span>

In [8]:
GDP = read_GDP()
sten = find_start_end()

bottoms = list()
for i in range(0, sten.shape[0]):
    st = sten.ix[i, 0]
    end = sten.ix[i, 1]
    
    # Must use np.logical_and() to compute the truth value of 2 conditions.
    sub_df = GDP.where(np.logical_and(GDP.quarter >= st, GDP.quarter <= end)).dropna()
    bottom_obs = (sub_df
                  .where(sub_df.billions_2009dollars == sub_df.billions_2009dollars.min())
                  .dropna()
                  .reset_index()) 
    
    # Must reset index in the previous step so that in the following step we can extract
    # the quarter value by .loc[0, 'quarter'] that returns only a string without any 
    # additional info of this observation.
    bottom_quarter = bottom_obs.loc[0, 'quarter']
    bottoms.append(bottom_quarter)
    
bottoms

['2009q2']

<br>

### Part (1):
Returns a DataFrame of towns and the states they are in from the university_towns.txt list.   
The following cleaning needs to be done:   
1. For "State", removing characters from "[" to the end.
2. For "RegionName", when applicable, removing every character from " (" to the end.
3. Depending on how you read the data, you may need to remove newline character '\n'. 

The format of the DataFrame should be like the following:

In [9]:
pd.DataFrame([["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"]],
             columns=["State", "RegionName"])

,State,RegionName
0,Michigan,Ann Arbor
1,Michigan,Yipsilanti


In [10]:
def get_list_of_university_towns():
    fh = open("university_towns.txt")

    # Use [] to create a list storing the content of the text file.
    lines = [line.strip() for line in fh.readlines()] 

    dt = pd.DataFrame(columns = ["State", "RegionName"])

    for i in range(0, len(lines)):
        line = lines[i]
        # re.match returns boolean values
        m = re.match("(.+)\[edit\]", line)  

        if m:
            # match.group([group1, ...]) returns one or more subgroups of the match.
            state = m.group(1)
        else:
            # There could be several blank lines at the end of the text file.
            if len(line) == 0:
                break
            line = re.sub(r' \(.*', '', line)
            dt = dt.append({'State': state, 'RegionName': line}, ignore_index = True)

    return dt

In [11]:
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


<br>
### Part (2):

Returns the <span style = 'color:red'>year and quarter</span> of the recession <span style = 'color:red'>start time</span> as a string value in a format such as 2005q3.

In [12]:
def get_recession_start():
    recess_start = find_start_end().loc[0, 'start_quarter']
    return recess_start

In [13]:
get_recession_start()

'2008q3'

<br>
### Part (3):

Returns the <span style = 'color:red'>year and quarter</span> of the recession <span style = 'color:red'>end time</span> as a string value in a format such as 2005q3.

In [14]:
def get_recession_end():
    recess_end = find_start_end().loc[0, 'end_quarter']
    return recess_end

In [15]:
get_recession_end()

'2009q4'

<br>
### Part (4):

Returns the <span style = 'color:red'>year and quarter</span> of the recession <span style = 'color:red'>bottom</span> time as a string value in a format such as 2005q3.

In [16]:
def get_recession_bottom():
    
    GDP = read_GDP()
    sten = find_start_end()

    for i in range(0, sten.shape[0]):
        st = sten.ix[i, 0]
        end = sten.ix[i, 1]

        # Must use np.logical_and() to compute the truth value of 2 conditions.
        sub_df = GDP.where(np.logical_and(GDP.quarter >= st, GDP.quarter <= end)).dropna()
        bottom_obs = (sub_df
                      .where(sub_df.billions_2009dollars == sub_df.billions_2009dollars.min())
                      .dropna()
                      .reset_index()) 

        # Must reset index in the previous step so that in the following step we can extract
        # the quarter value by .loc[0, 'quarter'] that returns only a string without any 
        # additional info of this observation.
        bottom_quarter = bottom_obs.loc[0, 'quarter']

    return bottom_quarter

In [17]:
get_recession_bottom()

'2009q2'

<br>

### Part (5):

Converts the housing data <span style = 'color:red'>to quarters</span> and returns it as <span style = 'color:red'>mean values</span> in a dataframe. This dataframe should be a dataframe with columns for <span style = 'color:red'>2000q1 through 2016q3</span>, and should have a multi-index in the shape of <span style = 'color:red'>["State","RegionName"]</span>.
    
Note: Quarters are defined in the assignment description, they are not arbitrary three month periods. 

The resulting dataframe should have 67 columns, and 10,730 rows. 

In [18]:
def convert_housing_data_to_quarters():

    # Extract the eligible months from the csv file to read. Then add the State and 
    # RegionName columns.
    eli_cols = pd.read_csv('City_Zhvi_AllHomes.csv', header = None, nrows=1).loc[0]
    eli_cols = (eli_cols
                .where(np.logical_and(eli_cols >= '2000-01', eli_cols <= '2016-09'))
                .dropna()
                .tolist())
    # Use extend() rather than append() to add 2 column names into the list.
    # Note: Do NOT assign this back to eli_cols.
    eli_cols.extend(['State', 'RegionName']) 

    # Prepare a dataframe of state abbreviation mapping.
    state_abbr = pd.DataFrame([states], columns = list(states.keys())).T
    
    # Load the csv file, reading only the eligible columns and convert the state 
    # abbreviations into their full names.
    housing = pd.read_csv('City_Zhvi_AllHomes.csv', usecols = eli_cols)
    housing = (housing.merge(state_abbr, left_on = 'State', right_index = True)
                      .drop('State', axis = 1)
                      .rename(columns = {0: 'State'})
                      .set_index(['State', 'RegionName']))


    # Add one last quarter into the quarter list.
    quarters = read_GDP().quarter.tolist()
    quarters.append('2016q3')

    
    # Calculate mean price for each quarter and store it in the corresponding new column.
    # The last quarter has only 2 months of data and need special processing. 
    
    # Cannot use range(0, 200, 3) because when i = 198, we'll have i + 3 = 201 and 
    # housing.iloc[ : , i : i + 3] would include columns "2016-07", "2016-08" AND THE 
    # ADDED-FROM-THE-FIRST-LOOP new column "2000q1" because we're adding one column into 
    # the housing data in each loop.
    i = 0
    for i in range(0, 200 - 2, 3):
        housing[quarters[int(i/3)]] = housing.iloc[ : , i : i + 3].mean(axis = 1)
        
    housing[quarters[-1]] = housing.iloc[ : , 198 : 200].mean(axis = 1)

    housing_quarterly = housing.iloc[ : , -67: ]

    return housing_quarterly

In [19]:
convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State    RegionName                                                             
New York New York                           NaN            NaN            NaN   
         Rochester                 53566.666667   53233.333333   53200.000000   
         Yonkers                  222233.333333  230900.000000  245533.333333   
         Syracuse                  56800.000000   57866.666667   57400.000000   
         Town of Greece            95100.000000   95500.000000   96333.333333   
         Albany                    91566.666667   92100.000000   91233.333333   
         Town of Poughkeepsie     131966.666667  135433.333333  139566.666667   
         New Rochelle             362766.666667  370866.666667  382233.333333   
         Town of Colonie          110600.000000  112400.000000  113166.666667   
         Mount Vernon             239333.333333  247300.000000  258633.333333   
         Utica                     50700.000000   51233.333333   52600.000000   
         Valley Stream            223166.666667  225166.666667  229133.333333   
         Schenectady               72266.666667   73100.000000   73733.333333   
         White Plains             284333.333333  297833.333333  314366.666667   
         Town of Hempstead        170166.666667  172266.666667  176800.000000   
         Town of Binghamton        81633.333333   82500.000000   81700.000000   
         Town of Newburgh         116000.000000  118400.000000  120233.333333   
         Troy                      74733.333333   76533.333333   78866.666667   
         Niagara Falls             50300.000000   50133.333333   50433.333333   
         Irondequoit               83400.000000   83500.000000   84400.000000   
         Binghamton                51766.666667   52000.000000   53366.666667   
         Massapequa               248833.333333  251233.333333  254500.000000   
         Town of Elmira            51200.000000   51600.000000   51700.000000   
         Henrietta                 94566.666667   94400.000000   95766.666667   
         Town of Webster          126433.333333  128600.000000  130833.333333   
         Patchogue                134400.000000  135200.000000  141966.666667   
         Clifton Park             140633.333333  142500.000000  143466.666667   
         Freeport                 183833.333333  186433.333333  190400.000000   
         Lindenhurst              168733.333333  169100.000000  173033.333333   
         Levittown                189966.666667  193366.666667  197866.666667   
...                                         ...            ...            ...   
Vermont  Plainfield                         NaN            NaN            NaN   
         West Brattleboro                   NaN            NaN            NaN   
         Duxbury                            NaN            NaN            NaN   
         Rochester                          NaN            NaN            NaN   
         Sharon                             NaN            NaN            NaN   
         White River Junction               NaN            NaN            NaN   
         Irasburg                           NaN            NaN            NaN   
         Mt Holly                           NaN            NaN            NaN   
         Ferrisburgh                        NaN            NaN            NaN   
         Lincoln                            NaN            NaN            NaN   
         Franklin                           NaN            NaN            NaN   
         Bridport                           NaN            NaN            NaN   
         Fayston                            NaN            NaN            NaN   
         Tunbridge                          NaN            NaN            NaN   
         Ryegate                            NaN            NaN            NaN   
         Shaftsbury                         NaN            NaN            NaN   
         Shoreham                           NaN            NaN          

<br>

### Part (6):

First create new data showing the decline or growth of housing prices <span style = 'color:red'>between the recession start and the recession bottom</span>. Then run a t test comparing the university town values to the non-university towns values, return whether the alternative hypothesis (that the two groups are the same) is <span style = 'color:red'>true or not</span> as well as the <span style = 'color:red'>p-value</span> of the confidence. 
    
Return the <span style = 'color:red'>tuple (different, p, better)</span> where different = True if the t-test is True at a p < 0.01 (we reject the null hypothesis), or different = False if otherwise (we cannot reject the null hypothesis). <span style = 'color:red'>The variable p should be equal to the exact p value returned from scipy.stats.ttest_ind().</span> The value for better should be either "university town" or "non-university town" depending on which has a <span style = 'color:red'>lower mean price ratio</span> (which is equivilent to a reduced market loss).

In [20]:
def run_ttest():
    
# Complete the quarter sequence (not necessary though).
    quarters = read_GDP().quarter
    quarters = quarters.append(pd.Series({len(quarters):'2016q3'}))

# Get the pre-recession quarter and the recession bottom quarter.
    recess_sten = find_start_end()
    st = recess_sten.loc[0, 'start_quarter']
    pre_quarter = quarters.iloc[quarters[quarters == st].index - 1].tolist()[0]
    bottom = get_recession_bottom()
    
# Get the housing dataframe and compute the mean price ratio for each town.
    hdf = convert_housing_data_to_quarters().loc[ :, [pre_quarter, bottom]].dropna()
    hdf['mean_price_ratio'] = hdf[pre_quarter].div(hdf[bottom])

# Split the housing dataset into university town set and non university town set.
    uni_town_names = get_list_of_university_towns()
    # convert the university towns (including the state and the region) to a list of tuples
    # before passing it to loc.
    uni_list_of_tuples = list(zip(uni_town_names.State, uni_town_names.RegionName))
    # Some university towns don't have any housing price data, but hdf.loc[uni_list_of_tuples]
    # would add those towns into the hdf dataframe index and fill the corresponding value with 
    # NaN. That's why we need to dropna() again.
    uni_towns_df = hdf.loc[uni_list_of_tuples].dropna()                         
    non_uni_towns_df = hdf[ ~ hdf.index.isin(uni_list_of_tuples)]

# Prepare the final answer.
    better = ("university town" 
              if uni_towns_df['mean_price_ratio'].mean() <= 
                 non_uni_towns_df['mean_price_ratio'].mean() 
              else "non-university town")
    ttest_result = ttest_ind(non_uni_towns_df['mean_price_ratio'], 
                             uni_towns_df['mean_price_ratio'])
    different = True if ttest_result.pvalue < 0.01 else False

    answer = tuple((different, ttest_result.pvalue, better))


    return answer

In [21]:
run_ttest()

(True, 0.0027240637047608778, 'university town')